In [4]:
import json
import logging
import requests
import traceback
import pandas as pd
from time import sleep

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

In [3]:
def getStockList(url):
    df = pd.read_html(url,encoding='big5hkscs',header=0)[0]
    stock_list = []
    stock_flag = False
    for row in df.iterrows():
        stock_no = row[1]['有價證券代號及名稱'].split('\u3000')[0]
        if stock_no == '股票':
            stock_flag = True
            continue
        elif stock_no == '上市認購(售)權證':
            stock_flag = False
            break
        if stock_flag:
            stock_list.append(stock_no[:4])
    return sorted(stock_list)

logging.info('get stock list')
twse_url = 'http://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
stock_list = getStockList(twse_url)

logging.info('get stock price')
final_result = []
query = ''
for i in range(len(stock_list)):
    if i % 150 == 0 and query != '':
        resp = requests.get('https://mis.twse.com.tw/stock/api/getStockInfo.jsp?json=1&delay=0&ex_ch=' + query[:-1])
        final_result.append(resp.text)
        query = ''
        sleep(3)
    query += 'tse_%s.tw|' % stock_list[i]
resp = requests.get('https://mis.twse.com.tw/stock/api/getStockInfo.jsp?json=1&delay=0&ex_ch=' + query[:-1])
final_result.append(resp.text)

logging.info('parse data')
stock_dict = {}
for result in final_result:
    for json_obj in json.loads(result).get('msgArray'):
        stock_dict[json_obj.get('c')] = json_obj.get('h')
print(stock_dict)

2020-07-06 14:17:28,870 [INFO] get stock list
2020-07-06 14:17:32,410 [INFO] get stock price
2020-07-06 14:18:15,021 [INFO] parse data


{'1101': '44.5000', '1102': '45.6000', '1103': '18.1500', '1104': '18.4000', '1108': '10.1000', '1109': '18.5500', '1110': '16.5000', '1201': '23.3000', '1203': '30.2000', '1210': '43.7500', '1213': '6.1000', '1215': '69.1000', '1216': '73.8000', '1217': '7.6600', '1218': '23.9500', '1219': '22.0500', '1220': '11.3000', '1225': '38.1000', '1227': '65.9000', '1229': '44.3000', '1231': '44.2000', '1232': '125.0000', '1233': '38.2000', '1234': '33.9500', '1235': '26.0500', '1236': '12.7500', '1256': '213.0000', '1301': '84.8000', '1303': '67.0000', '1304': '12.5000', '1305': '19.5000', '1307': '21.8500', '1308': '19.4500', '1309': '15.3000', '1310': '16.9000', '1312': '15.0500', '1313': '10.2500', '1314': '8.4500', '1315': '53.3000', '1316': '12.5000', '1319': '37.9500', '1321': '32.7000', '1323': '33.8000', '1324': '9.8000', '1325': '163.5000', '1326': '73.8000', '1337': '5.1000', '1338': '57.6000', '1339': '46.6000', '1340': '10.3500', '1341': '60.2000', '1402': '29.1500', '1409': '11.8